In [ ]:
import pandas as pd
import pubchempy as pcp
import requests
from bs4 import BeautifulSoup
import csv

# Загрузка данных из файла CSV
filename = 'C:/Users/vasae/PycharmProjects/pythonProject1/7.csv'

# Создание нового файла с колонкой "Name"
df = pd.read_csv(filename, usecols=['Name'], encoding='unicode_escape')
df.to_csv('7end.csv', index=False, encoding='utf-8')
# Обработка данных в партиях по 100 строк
for start_idx in range(0, len(df), 100):
        end_idx = min(start_idx + 100, len(df))
        df_batch = df.iloc[start_idx:end_idx]
        # Поиск CID и IUPAC Name по колонке "Name"
        for index, row in df_batch.iterrows():
            try:
                name_str = str(row['Name'])
                # Поиск CID
                cids = pcp.get_cids(name_str, 'name')
                if cids:
                    cid = cids[0]
                else:
                    # Поиск CID на сайте PubChem
                    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name_str}/cids/JSON'
                    response = requests.get(url)
                    response_json = response.json()
                    if 'IdentifierList' in response_json:
                        cid = response_json['IdentifierList']['CID'][0]
                    else:
                        cid = ''
                # Поиск IUPAC Name по CID
                if cid:
                    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/IUPACName/txt'
                    response = requests.get(url)
                    if response.status_code == 200:
                        iupac_name = response.text.strip()
                    else:
                        iupac_name = ''
                else:
                    # Поиск IUPAC Name по имени на сайте Cactus NCI
                    url = f'https://cactus.nci.nih.gov/chemical/structure/{name_str}/iupac_names'
                    response = requests.get(url)
                    soup = BeautifulSoup(response.content, 'html.parser')
                    iupac_name = soup.find('pre').text.strip() if soup.find('pre') else ''
                    if not iupac_name:
                        # Поиск IUPAC Name по имени на сайте PubChem
                        url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name_str}/property/IUPACName/txt'
                        response = requests.get(url)
                        iupac_name = response.text.strip() if response.status_code == 200 else ''
                # Запись обновленных данных в выходной файл
                df.loc[index, 'CID'] = cid
                df.loc[index, 'IUPAC Name'] = iupac_name
            except:
                pass
        # Запись результатов текущей партии в выходной файл
        df.iloc[start_idx:end_idx].to_csv('temp7.csv', mode='a', header=False, index=False, encoding='utf-8')
    # Замена временного файла выходным файлом
os.replace('temp7.csv', '7end.csv')


In [2]:
import pandas as pd
from pubchempy import get_compounds
from urllib.parse import quote
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time

# Загрузка данных из файла
filename = '6result.xlsx'
df = pd.read_excel(filename)

# Запросы по поиску нотации SMILES для отсутствующих значений
for i, row in df.iterrows():
    smiles = row['smiles']
    iupac_name = row['iupac_name']
    name = row['name']
    
    if pd.isna(smiles):
        if pd.notna(iupac_name):
            # Иначе, если IUPAC Name есть, пробуем найти по нему
            url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{iupac_name}/property/IsomericSMILES/txt'
            try:
                response = urlopen(url)
                smiles = response.read().decode('utf-8').strip()
            except:
                smiles = ''
        
        if pd.isna(iupac_name) or not smiles:
            # Если IUPAC Name отсутствует или поиск по нему не дал результатов, пробуем найти по имени
            url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name}/property/IsomericSMILES/txt'
            try:
                response = urlopen(url)
                smiles = response.read().decode('utf-8').strip()
            except:
                smiles = '' 

        if not smiles:
            if pd.notna(iupac_name):
                # Если результат не был найден ни по IUPAC Name, ни по имени, пробуем получить SMILES через Cactus NCI
                url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(iupac_name) + '/smiles'
                try:
                    smiles = urlopen(url).read().decode('utf-8').strip()
                except:
                    smiles = ''
            if pd.isna(iupac_name) or not smiles:
                url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(name) + '/smiles'
                try:
                    smiles = urlopen(url).read().decode('utf-8').strip()
                except:
                    smiles = ''
        df.at[i, 'smiles'] = smiles
    time.sleep(1)

# Сохранение результатов в исходный файл
df.to_excel(filename, index=False)


In [4]:
import pandas as pd
import requests

problematic_doi = []

# Replace the filename with the path to your Excel file
filename = 'C:/Users/vasae/333.xlsx'

# Load the Excel file into a pandas dataframe
df = pd.read_excel(filename)

# Iterate over the dataframe rows and retrieve metadata for each DOI
for index, row in df.iterrows():
    # Check if any of the cells in the "Date", "Journal", or "Title" columns is empty
    if pd.isna(row["Date"]) or pd.isna(row["Journal"]) or pd.isna(row["Title"]):
        doi = row["DOI"]
        response = requests.get(f"https://api.crossref.org/works/{doi}")
        if response.ok:
            data = response.json()["message"]
            # Fill in empty cells in the "Date", "Journal", and "Title" columns
            if pd.isna(row["Date"]):
                df.at[index, "Date"] = data["created"]["date-parts"][0][0]
            if pd.isna(row["Journal"]):
                df.at[index, "Journal"] = data["container-title"][0]
            if pd.isna(row["Title"]):
                df.at[index, "Title"] = data["title"][0]
            print(f"DOI: {data['DOI']}")
        else:
            problematic_doi.append(doi)
            print("DOI not found.")

# Save the modified dataframe back to the Excel file
df.to_excel(filename, index=False)


DOI: 10.1016/s0168-583x(03)00815-2
DOI: 10.1039/c5nj03339j
DOI: 10.1039/c3pp50219h
DOI: 10.1016/j.ceramint.2003.09.010
DOI: 10.1016/s0032-3861(02)00727-9
DOI: 10.1039/c8nj03435d
DOI: 10.1016/s0040-6090(02)00294-8
DOI: 10.1016/j.spmi.2004.08.035
DOI: 10.1039/c4cp04816d
DOI: 10.1016/j.physe.2016.01.001
DOI: 10.1016/j.jcis.2017.10.101
DOI: 10.1039/c6nr03313j
DOI: 10.1039/c5py01920f
DOI: 10.1016/j.optcom.2016.04.061
DOI: 10.1039/b506399j
DOI: 10.1016/s0169-4332(99)00524-3
DOI: 10.1039/b803052a
DOI: 10.1016/j.sna.2018.02.021
DOI: 10.1016/j.optcom.2004.09.016
DOI: 10.1039/c8lc00356d
DOI: 10.1039/c1jm13955j
DOI: 10.1016/j.optcom.2016.04.061
DOI: 10.1016/s0921-4526(99)00708-5
DOI: 10.1016/j.apsusc.2003.12.028
DOI: 10.1039/c8nr08544g
DOI: 10.1016/s0168-9002(03)00789-7
DOI: 10.1039/c6ra21016c
DOI: 10.1038/s41598-017-00726-3
DOI: 10.1186/1556-276x-9-658
DOI: 10.1039/c5tc00765h
DOI: 10.1039/c0gc00853b
DOI: 10.1016/j.tsf.2003.09.055
DOI: 10.1007/s10762-013-0001-8
DOI: 10.1016/j.bios.2015.07.022
DOI

DOI: 10.1039/c6ra20355h
DOI: 10.1039/c2lc40406k
DOI: 10.1016/j.sna.2004.01.015
DOI: 10.1039/c6tc00663a
DOI: 10.1039/b414064h
DOI: 10.1007/s13204-015-0493-8
DOI: 10.1016/j.solener.2015.10.003
DOI: 10.1039/c7ob00205j
DOI: 10.1039/c2ee22952h
DOI: 10.1039/c7tb00777a
DOI: 10.1016/j.jlumin.2014.12.041
DOI: 10.1016/s0927-0248(02)00083-1
DOI: 10.1016/j.cplett.2004.09.134
DOI: 10.1038/s41598-017-17822-z
DOI: 10.1039/b822980p
DOI: 10.1039/c3ay40952j
DOI: 10.1039/c3sm52279b
DOI: 10.1016/j.ijleo.2015.03.029
DOI: 10.1039/c7fd00008a
DOI: 10.1039/c2lc40616k
DOI: 10.1016/s0379-6779(01)00643-9
DOI: 10.1016/j.cjph.2016.08.002
DOI: 10.1016/s0927-0248(02)00473-7
DOI: 10.1016/j.matchemphys.2018.03.077
DOI: 10.1038/s41598-017-10712-4
DOI: 10.1007/s11665-013-0516-0
DOI: 10.1039/c8ay00491a
DOI: 10.1039/b212337a
DOI: 10.1039/c6nr01737a
DOI: 10.1016/s0925-3467(01)00040-4
DOI: 10.1016/j.yofte.2018.05.007
DOI: 10.1039/c4ra09579k
DOI: 10.1039/c5nr09061j
DOI: 10.1016/j.optlastec.2017.12.010
DOI: 10.1016/s0953-4431(